<a href="https://colab.research.google.com/github/davidwhogg/synthetic_calibration/blob/main/ipynb/eboss_arcs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploratory data analysis with *SDSS-IV eBOSS* arc exposures

*by* **David W. Hogg**



In [ ]:
import numpy as np
import pylab as plt
from matplotlib.backends.backend_pdf import PdfPages
from astropy.io import fits
from urllib.request import Request, urlopen, urlretrieve
from bs4 import BeautifulSoup

In [ ]:
def make_boss_data_url(mjd):
  return "https://data.sdss.org/sas/dr16/eboss/spectro/data/{:05d}/".format(mjd)

def get_all_sdR_r1_urls(url):
  """
  This code is overly specific!
  """
  req = Request(url)
  a = urlopen(req).read()
  soup = BeautifulSoup(a, 'html.parser')
  x = (soup.find_all('a'))
  urls = []
  for i in x:
    file_name = i.extract().get_text()
    if "sdR-r1-" in file_name:
      url_new = url + file_name
      # url_new = url_new.replace(" ","%20")
      urls.append(url_new)
  return np.unique(urls)

def grab_all_arcs(mjd):
  url = make_boss_data_url(mjd)
  sdR_urls = get_all_sdR_r1_urls(url)
  arcs = []
  for url in sdR_urls:
    hdu = fits.open(url)
    if "arc" in hdu[0].header["FLAVOR"]:
      print("found arc", url)
      arcs.append((url, hdu[0].header, hdu[0].data))
    hdu.close()
    del hdu
  print("found a total of {} arcs".format(len(arcs)))
  return arcs

In [ ]:
arcs = []
for mjd in (58539, 58541):
  arcs += grab_all_arcs(mjd)

In [ ]:
for url, hdr, image in arcs:
  print(image.size, np.min(image), np.median(image), np.max(image))


In [ ]:
with PdfPages('boss_arc_zoom.pdf') as pdf:
  for url, hdr, image in arcs:
    subimage = image[1700:1900, 1500:1700].astype(float)
    subimage -= np.median(subimage)
    if (np.max(subimage) - np.min(subimage)) > 30.:
      fig = plt.figure(figsize=(12, 9))
      plt.imshow(subimage, vmin=-30, vmax=30.0,
                 cmap="gray", interpolation="nearest")
      plt.colorbar()
      plt.title(url)
      pdf.savefig(fig)
      plt.close(fig)

In [ ]:
def _cc(dx, dy, image, reference, maxshift):
  resid = reference * np.roll(image, (dy, dx), axis=(0, 1))
  return np.sum(resid[maxshift:-maxshift,maxshift:-maxshift])

def cross_correlate(image, reference, maxshift=20):
  assert image.shape == reference.shape
  shifts = np.arange(-maxshift, maxshift + 1)
  ccf = np.zeros((len(shifts), len(shifts)))
  for ix, dx in enumerate(shifts):
    for iy, dy in enumerate(shifts):
      ccf[iy, ix] = _cc(dx, dy, image, reference, maxshift)
  return ccf, shifts

In [ ]:
def imslice(image):
  return (image[1700:1900, 1500:1700]).astype(float)

refimage = imslice(arcs[0][2])
refimage -= np.median(refimage)
with PdfPages('boss_arc_ccf.pdf') as pdf:
  for url, hdr, image in arcs:
    subimage = imslice(image)
    subimage -= np.median(subimage)
    ccf, shifts = cross_correlate(subimage, refimage)
    ismax = np.where(ccf == np.max(ccf))
    fig = plt.figure(figsize=(12, 9))
    plt.imshow(ccf,
                 cmap="gray", interpolation="nearest")
    plt.colorbar()
    plt.title(url + " ({},{})".format(shifts[ismax[0][0]], shifts[ismax[1][0]]))
    pdf.savefig(fig)
    plt.close(fig)